In [2]:
import pandas as pd
import glob
import os
import re
from tqdm import tqdm


In [3]:
data_dir = "C:/Users/Elaine/Desktop/TCC_acidentes/data/" 

# 1 ABERTURA DOS DADOS

In [4]:
caminho = fr'{data_dir}\ac_por_pessoa\*.csv'
arquivos = glob.glob(caminho)

print(arquivos)
print(len(arquivos), "arquivos encontrados")

['C:/Users/Elaine/Desktop/TCC_acidentes/data/\\ac_por_pessoa\\acidentes2019_todas_causas_tipos.csv', 'C:/Users/Elaine/Desktop/TCC_acidentes/data/\\ac_por_pessoa\\acidentes2020_todas_causas_tipos.csv', 'C:/Users/Elaine/Desktop/TCC_acidentes/data/\\ac_por_pessoa\\acidentes2021_todas_causas_tipos.csv']
3 arquivos encontrados


## 1.1 PRF

In [7]:
# Caminho onde estão os arquivos de acidentes
path_acidentes = fr'{data_dir}\ac_por_pessoa\*.csv'

lista_acidentes = []

for arquivo in tqdm(glob.glob(path_acidentes)):
    
    # Lê o CSV
    df = pd.read_csv(arquivo, encoding="ISO-8859-1", sep=";")
    
    # Filtrar colunas que você precisa
    df = df[['uf', 'br', 'km', 'municipio', 'dia_semana', 'fase_dia', 'sentido_via',
             'condicao_metereologica', 'tipo_pista', 'tracado_via', 'uso_solo',
             'tipo_veiculo', 'data_inversa', 'estado_fisico']]
    
    # Converte data
    df["data_inversa"] = pd.to_datetime(df["data_inversa"], errors="coerce")
    
    # Extrair ano
    df["ano"] = df["data_inversa"].dt.year
    
    lista_acidentes.append(df)

# Concatena tudo em um dataframe só
prf = pd.concat(lista_acidentes, ignore_index=True)
prf.info()
prf.head()

100%|██████████| 3/3 [00:16<00:00,  5.60s/it]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1145355 entries, 0 to 1145354
Data columns (total 15 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   uf                      1145355 non-null  object        
 1   br                      1142804 non-null  float64       
 2   km                      1142804 non-null  object        
 3   municipio               1145355 non-null  object        
 4   dia_semana              1145355 non-null  object        
 5   fase_dia                1145355 non-null  object        
 6   sentido_via             1145355 non-null  object        
 7   condicao_metereologica  1145355 non-null  object        
 8   tipo_pista              1145355 non-null  object        
 9   tracado_via             1145355 non-null  object        
 10  uso_solo                1145355 non-null  object        
 11  tipo_veiculo            1101989 non-null  object        
 12  data_inversa  

,uf,br,km,municipio,dia_semana,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,tipo_veiculo,data_inversa,estado_fisico,ano
0,SP,116.0,218,GUARULHOS,terça-feira,Plena Noite,Crescente,Céu Claro,Múltipla,Curva,Sim,Automóvel,2019-01-01,Lesões Leves,2019
1,SP,116.0,218,GUARULHOS,terça-feira,Plena Noite,Crescente,Céu Claro,Múltipla,Curva,Sim,Automóvel,2019-01-01,Lesões Leves,2019
2,SP,116.0,218,GUARULHOS,terça-feira,Plena Noite,Crescente,Céu Claro,Múltipla,Curva,Sim,Automóvel,2019-01-01,Lesões Leves,2019
3,SP,116.0,218,GUARULHOS,terça-feira,Plena Noite,Crescente,Céu Claro,Múltipla,Curva,Sim,Automóvel,2019-01-01,Lesões Leves,2019
4,PR,373.0,"177,3",PONTA GROSSA,terça-feira,Plena Noite,Crescente,Nublado,Dupla,Reta,Sim,Motoneta,2019-01-01,Lesões Leves,2019


## 1.2 Frota

In [8]:
path_frota = fr'{data_dir}\frota\*.xls'

lista_frotas = []

def extrair_ano_primeira_linha(arquivo):
    """
    Lê apenas a primeira linha do arquivo XLS como texto
    e extrai o ano (4 dígitos) que aparece no final.
    """
    # lê SOMENTE a primeira linha como texto bruto
    df_tmp = pd.read_excel(arquivo, header=None, nrows=1, dtype=str)
    linha = str(df_tmp.iloc[0,0])

    # pega 4 dígitos no final
    ano = re.findall(r'(\d{4})', linha)
    if ano:
        return int(ano[-1])
    else:
        raise ValueError(f"Não foi possível extrair o ano do arquivo: {arquivo}")

# leitura das tabelas de frota
for arquivo in tqdm(glob.glob(path_frota)):

    ano = extrair_ano_primeira_linha(arquivo)

    # agora lê normalmente, pulando cabeçalho
    df = pd.read_excel(arquivo, skiprows=3)

    df = df.rename(columns={"UF": "uf", "MUNICIPIO": "municipio", "TOTAL": "frota"})
    df["ano"] = ano

    # mantém apenas o necessário
    df = df[["uf", "municipio", "frota", "ano"]]

    lista_frotas.append(df)

# concatena
frota = pd.concat(lista_frotas, ignore_index=True)
frota.head()

100%|██████████| 3/3 [00:08<00:00,  2.83s/it]


,uf,municipio,frota,ano
0,AC,ACRELANDIA,7204,2021
1,AC,ASSIS BRASIL,2075,2021
2,AC,BRASILEIA,11294,2021
3,AC,BUJARI,2442,2021
4,AC,CAPIXABA,2497,2021


# 2 ACIDENTES E FROTA

In [9]:
ac_frota = prf.merge(
    frota,
    on=["municipio", "uf", "ano"],
    how="left"
)

ac_frota['gravidade'] = ac_frota['estado_fisico'].apply(
    lambda x: 1 if x in ['Lesões Graves', 'Óbito'] else 0
)

ac_frota['km'] = ac_frota['km'].str.replace(",", ".").astype(float).round(0) #! arredondei os kms

del ac_frota['estado_fisico'], ac_frota['data_inversa'], ac_frota['municipio']

print(ac_frota['gravidade'].value_counts(normalize=True))
ac_frota.info()
ac_frota.head()

gravidade
0    0.841888
1    0.158112
Name: proportion, dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1145355 entries, 0 to 1145354
Data columns (total 14 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   uf                      1145355 non-null  object 
 1   br                      1142804 non-null  float64
 2   km                      1142804 non-null  float64
 3   dia_semana              1145355 non-null  object 
 4   fase_dia                1145355 non-null  object 
 5   sentido_via             1145355 non-null  object 
 6   condicao_metereologica  1145355 non-null  object 
 7   tipo_pista              1145355 non-null  object 
 8   tracado_via             1145355 non-null  object 
 9   uso_solo                1145355 non-null  object 
 10  tipo_veiculo            1101989 non-null  object 
 11  ano                     1145355 non-null  int32  
 12  frota                   1132257 non-null 

,uf,br,km,dia_semana,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,tipo_veiculo,ano,frota,gravidade
0,SP,116.0,218.0,terça-feira,Plena Noite,Crescente,Céu Claro,Múltipla,Curva,Sim,Automóvel,2019,705649.0,0
1,SP,116.0,218.0,terça-feira,Plena Noite,Crescente,Céu Claro,Múltipla,Curva,Sim,Automóvel,2019,705649.0,0
2,SP,116.0,218.0,terça-feira,Plena Noite,Crescente,Céu Claro,Múltipla,Curva,Sim,Automóvel,2019,705649.0,0
3,SP,116.0,218.0,terça-feira,Plena Noite,Crescente,Céu Claro,Múltipla,Curva,Sim,Automóvel,2019,705649.0,0
4,PR,373.0,177.0,terça-feira,Plena Noite,Crescente,Nublado,Dupla,Reta,Sim,Motoneta,2019,220391.0,0


In [10]:
amostra = ac_frota.sample(frac=0.10, random_state=42)

# 3 EXPORT

In [11]:
ac_frota.to_csv(os.path.join(data_dir, "tratados/acidentes_frota_19-21.csv"), index=False)
amostra.to_csv(os.path.join(data_dir, "tratados/amostra_19-21.csv"), index=False)